In [1]:
import gradio as gr
import pandas as pd
import joblib

In [2]:
vectorizer = joblib.load('models/tfidf_vectorizer.pkl')
kmeans_model = joblib.load('models/kmeans_model.pkl')
processed_data = pd.read_csv('data/processed_data/final_data.csv')

In [3]:
def recommend_products(user_id):
    user_data = processed_data[processed_data['UserId'] == user_id]
    if user_data.empty:
        return "❌ User ID not found!"
    
    user_text = " ".join(user_data['Cleaned_Text'].values)
    user_vector = vectorizer.transform([user_text])
    
    user_cluster = kmeans_model.predict(user_vector)
    
    cluster_products = processed_data[processed_data['Cluster'] == user_cluster[0]]
    recommendations = cluster_products['ProductId'].value_counts().head(5).index.tolist()
    
    return "\n".join(recommendations)

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # 🛒 E-commerce Product Recommendation Engine
    Welcome! Enter your **User ID** to get your top 5 product recommendations based on Amazon Fine Food Reviews!
    """)

    with gr.Row():
        with gr.Column():
            user_input = gr.Textbox(label="🔎 Enter User ID", placeholder="e.g. A3SGXH7AUHU8GW")
            submit_btn = gr.Button("🎯 Recommend Products")
        with gr.Column():
            output = gr.Textbox(label="📦 Top 5 Recommended Product_ID", lines=5)

    submit_btn.click(fn=recommend_products, inputs=user_input, outputs=output)

demo.launch(share=True)


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://99d758ab0a3581d967.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
